# DeepSVR Experiment

## Select Dataset for Experiment

In [1]:
# Select 'housing' or 'space_ga'
dataset='housing'

## Load Standard Libraries

In [2]:
import numpy as np # General numerical operations
from sklearn.neural_network import MLPRegressor # Standard Neural Net
import tensorflow as tf # Deep Learning
from tensorflow.python.framework import ops # Tensorflow options
import pandas as pd # Data frame
import matplotlib.pyplot as plt # Plotting function
import sklearn.datasets as sk_dat;
from sklearn.utils import shuffle;
from sklearn.metrics import make_scorer, mean_absolute_error,accuracy_score, r2_score
from sklearn.model_selection import GridSearchCV;
from sklearn.pipeline import Pipeline;
from sklearn.preprocessing import StandardScaler;
from sklearn.svm import SVC, SVR;
from sklearn.base import (BaseEstimator, ClassifierMixin, RegressorMixin,TransformerMixin);
from sklearn.utils.validation import check_array, check_is_fitted, check_X_y;
from keras.layers import Dense, Input, TimeDistributed;
from keras.regularizers import l1 as l1_, l2 as l2_, l1_l2 as l1_l2_;
from keras.models import Model, load_model, save_model;
from keras.optimizers import (Adadelta, Adagrad, Adam, Adamax, Nadam, RMSprop,
SGD);
from keras import backend as K;
from keras.callbacks import EarlyStopping;
from sklearn.datasets import (get_data_home, load_svmlight_file,
                              load_svmlight_files);
from os.path import basename, exists, expanduser, join, normpath, splitext;
from os import environ, makedirs, remove;
from bz2 import decompress
from os import environ, makedirs, remove
from os.path import basename, exists, expanduser, join, normpath, splitext
from shutil import copyfileobj
from sklearn.datasets import (get_data_home, load_svmlight_file,
                              load_svmlight_files)
from sklearn.datasets.base import Bunch
from urllib.error import HTTPError
from urllib.request import urlopen
from zipfile import ZipFile
from sklearn.model_selection import cross_val_score;
from skopt import BayesSearchCV;

/home/falendor/.conda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


## Load Custom Libraries

In [3]:
class Architecture(BaseEstimator):

    """Architecture.
    Architecture class.
    Parameters
    ----------
    transformer: keras function, default=None
                 Feature transformation.
    activation: string/function, default='linear'/'softmax'
                Activation function to use.
    use_bias: boolean, default=True
              Whether the layer uses a bias vector.
    kernel_initializer: string/function, default='glorot_uniform'
                        Initializer for the kernel weights matrix.
    bias_initializer: string/function, default='zeros'
                      Initializer for the bias vector.
    kernel_regularizer_l1: float, default=None
                           L1 regularization factor applied to the kernel
                           weights matrix.
    kernel_regularizer_l2: float, default=None
                           L2 regularization factor applied to the kernel
                           weights matrix.
    bias_regularizer_l1: float, default=None
                         L1 regularization factor applied to the bias vector.
    bias_regularizer_l2: float, default=None
                         L2 regularization factor applied to the bias vector.
    activity_regularizer_l1: float, default=None
                             L1 regularization factor applied to the output of
                             the layer.
    activity_regularizer_l2: float, default=None
                             L2 regularization factor applied to the output of
                             the layer.
    kernel_constraint: function, default=None
                       Constraint function applied to the kernel weights matrix.
    bias_constraint: function, default=None
                     Constraint function applied to the bias vector.
    return_sequences: boolean, default=False
                      Whether to return the last output in the output sequence,
                      or the full sequence.
    Returns
    -------
    Model
    """

    def __new__(cls, X, y, transformer=None, activation='linear', use_bias=True,
                kernel_initializer='glorot_uniform', bias_initializer='zeros',
                kernel_regularizer_l1=None, kernel_regularizer_l2=None,
                bias_regularizer_l1=None, bias_regularizer_l2=None,
                activity_regularizer_l1=None, activity_regularizer_l2=None,
                kernel_constraint=None, bias_constraint=None,
                return_sequences=False):
        z = inputs = Input(shape=X.shape[1:])
        if transformer is not None: z = transformer(z)
        layer = Dense(int(np.prod(y.shape[1:])), activation=activation,
                      use_bias=use_bias,
                      kernel_initializer=kernel_initializer,
                      bias_initializer=bias_initializer,
                      kernel_regularizer=Regularizer(l1=kernel_regularizer_l1,
                                                     l2=kernel_regularizer_l2),
                      bias_regularizer=Regularizer(l1=bias_regularizer_l1,
                                                   l2=bias_regularizer_l2),
                      activity_regularizer=Regularizer(l1=activity_regularizer_l1,
                                                       l2=activity_regularizer_l2),
                      kernel_constraint=kernel_constraint,
                      bias_constraint=bias_constraint)
        if return_sequences: layer = TimeDistributed(layer)
        output = layer(z)
        return Model(inputs, output)


def _time_series(X, y=None, window=None, return_sequences=False):
    """Time series transformation.
    Transform X, y tensors to time series tensors.
    Parameters
    ----------
    X: numpy array of shape [n_samples, n_features]
       Training set.
    y: numpy array of shape [n_samples]
       Target values.
    window: integer, default=None
            Time window length.
    return_sequences: boolean, default=False
                      Whether to return the last output in the output sequence,
                      or the full sequence.
    Returns
    -------
    Time series tensors.
    """
    if window is not None:
        X = np.array([X[i:i + window] for i in range(X.shape[0] - window + 1)])
        if y is not None:
            y = np.array([y[i:i + window] for i in range(y.shape[0] - window + 1)]) if return_sequences else y[window - 1:]
    return X, y

class Optimizer():

    """Optimizer.
    Optimizer class.
    Parameters
    ----------
    optimizer: {"sgd", "rmsprop", "adagrad", "adadelta", "adam", "adamax",
               "nadam"}, default='adam'
               Optimizer
    lr: float>=0, default=0.001
        Learning rate.
    momentum: float>=0, default=0.0
              Parameter updates momentum.
    nesterov: boolean, default=False
              Whether to apply Nesterov momentum.
    decay: float>=0, default=0.0
           Learning rate decay over each update.
    rho: float>=0, default=0.9
    epsilon: float>=0, default=1e-08
             Fuzz factor.
    beta_1: float in (0, 1), default=0.9
    beta_2: float in (0, 1), default=0.999
    schedule_decay: , default=0.004
    Returns
    -------
    Optimizer
    """

    def __new__(cls, optimizer='adam', lr=0.001, momentum=0.0, nesterov=False,
                decay=0.0, rho=0.9, epsilon=1e-08, beta_1=0.9, beta_2=0.999,
                schedule_decay=0.004):
        optimizers = {'sgd':  SGD(lr=lr, momentum=momentum, decay=decay,
                                  nesterov=nesterov),
                      'rmsprop': RMSprop(lr=lr, rho=rho, epsilon=epsilon,
                                         decay=decay),
                      'adagrad': Adagrad(lr=lr, epsilon=epsilon, decay=decay),
                      'adadelta': Adadelta(lr=lr, rho=rho, epsilon=epsilon,
                                           decay=decay),
                      'adam': Adam(lr=lr, beta_1=beta_1, beta_2=beta_2,
                                   epsilon=epsilon, decay=decay),
                      'adamax': Adamax(lr=lr, beta_1=beta_1, beta_2=beta_2,
                                       epsilon=epsilon, decay=decay),
                      'nadam': Nadam(lr=lr, beta_1=beta_1, beta_2=beta_2,
                                     epsilon=epsilon,
                                     schedule_decay=schedule_decay)}
        return optimizers[optimizer]
    
class Regularizer():

    """Regularizer.
    Regularizer class.
    Parameters
    ----------
    l1: float, default=None
        L1 regularization factor.
    l2: float, default=None
        L2 regularization factor.
    Returns
    -------
    Regularizer.
    """

    def __new__(cls, l1=None, l2=None):
        if (l1 is None) and (l2 is not None):
            regularizer = l2_(l=l2)
        elif (l1 is not None) and (l2 is None):
            regularizer = l1_(l=l1)
        elif (l1 is not None) and (l2 is not None):
            regularizer = l1_l2_(l1=l1, l2=l2)
        else:
            regularizer = None
        return regularizer

class BaseFeedForward(BaseEstimator, TransformerMixin):

    """Feed-forward regressor/classifier.
    This model optimizes the MSE/categorical-crossentropy function using
    back-propagation.
    Parameters
    ----------
    transformer: keras function, default=None
                 Feature transformation.
    activation: string/function, default='linear'/'softmax'
                Activation function to use.
    use_bias: boolean, default=True
              Whether the layer uses a bias vector.
    kernel_initializer: string/function, default='glorot_uniform'
                        Initializer for the kernel weights matrix.
    bias_initializer: string/function, default='zeros'
                      Initializer for the bias vector.
    kernel_regularizer_l1: float, default=None
                           L1 regularization factor applied to the kernel
                           weights matrix.
    kernel_regularizer_l2: float, default=None
                           L2 regularization factor applied to the kernel
                           weights matrix.
    bias_regularizer_l1: float, default=None
                         L1 regularization factor applied to the bias vector.
    bias_regularizer_l2: float, default=None
                         L2 regularization factor applied to the bias vector.
    activity_regularizer_l1: float, default=None
                             L1 regularization factor applied to the output of
                             the layer.
    activity_regularizer_l2: float, default=None
                             L2 regularization factor applied to the output of
                             the layer.
    kernel_constraint: function, default=None
                       Constraint function applied to the kernel weights matrix.
    bias_constraint: function, default=None
                     Constraint function applied to the bias vector.
    optimizer: {"sgd", "rmsprop", "adagrad", "adadelta", "adam", "adamax",
               "nadam"}, default='adam'
               Optimizer
    lr: float>=0, default=0.001
        Learning rate.
    momentum: float>=0, default=0.0
              Parameter updates momentum.
    nesterov: boolean, default=False
              Whether to apply Nesterov momentum.
    decay: float>=0, default=0.0
           Learning rate decay over each update.
    rho: float>=0, default=0.9
    epsilon: float>=0, default=1e-08
             Fuzz factor.
    beta_1: float in (0, 1), default=0.9
    beta_2: float in (0, 1), default=0.999
    schedule_decay: , default=0.004
    loss: string/function, default='mse'/'categorical_crossentropy'
          Loss function.
    metrics: list, default=None
             List of metrics to be evaluated by the model during training and
             testing.
    loss_weights: list or dictionary, default=None
                  Scalar coefficients to weight the loss contributions of
                  different model outputs.
    sample_weight_mode: {"temporal", None}, default=None
                        Timestep-wise sample weighting.
    batch_size: integer, default='auto'
                Number of samples per gradient update.
    epochs: integer, default=200
            The number of times to iterate over the training data arrays.
    verbose: {0, 1, 2}, default=2
             Verbosity mode. 0=silent, 1=verbose, 2=one log line per epoch.
    early_stopping: bool, default True
                    Whether to use early stopping to terminate training when
                    validation score is not improving.
    tol: float, default 1e-4
         Tolerance for the optimization.
    patience: integer, default 2
              Number of epochs with no improvement after which training will
              be stopped.
    validation_split: float in [0, 1], default=0.1
                      Fraction of the training data to be used as validation
                      data.
    validation_data: array-like, shape ((n_samples, features_shape),
                                        (n_samples, targets_shape)),
                     default=None
                     Data on which to evaluate the loss and any model metrics at
                     the end of each epoch.
    shuffle: boolean, default=True
             Whether to shuffle the training data before each epoch.
    class_weight: dictionary, default=None
                  class indices to weights to apply to the model's loss for the
                  samples from each class during training.
    sample_weight: array-like, shape (n_samples), default=None
                   Weights to apply to the model's loss for each sample.
    initial_epoch: integer, default=0
                   Epoch at which to start training.
    window: integer, default=None
            Time window length.
    return_sequences: boolean, default=False
                      Whether to return the last output in the output sequence,
                      or the full sequence.
    """

    def __init__(self, transformer=None, activation='relu', use_bias=True,
                 kernel_initializer='glorot_uniform', bias_initializer='zeros',
                 kernel_regularizer_l1=None, kernel_regularizer_l2=None,
                 bias_regularizer_l1=None, bias_regularizer_l2=None,
                 activity_regularizer_l1=None, activity_regularizer_l2=None,
                 kernel_constraint=None, bias_constraint=None, optimizer='adam',
                 lr=0.001, momentum=0.0, nesterov=False, decay=0.0, rho=0.9,
                 epsilon=1e-08, beta_1=0.9, beta_2=0.999, schedule_decay=0.004,
                 loss='mse', metrics=None, loss_weights=None,
                 sample_weight_mode=None, batch_size='auto', epochs=200,
                 verbose=1, early_stopping=True, tol=0.0001, patience=2,
                 validation_split=0.1, validation_data=None, shuffle=True,
                 class_weight=None, sample_weight=None, initial_epoch=0,
                 window=None, return_sequences=False):
        for k, v in locals().items():
            if k != 'self': self.__dict__[k] = v

    def fit(self, X, y, optimizer=None, lr=None, momentum=None, nesterov=None,
            decay=None, rho=None, epsilon=None, beta_1=None, beta_2=None,
            schedule_decay=None, loss=None, metrics=None, loss_weights=None,
            sample_weight_mode=None, batch_size=None, epochs=None, verbose=1,
            early_stopping=None, tol=None, patience=None, validation_split=None,
            validation_data=None, shuffle=None, class_weight=None,
            sample_weight=None, initial_epoch=None):
        """Fit to data.
        Fit model to X.
        Parameters
        ----------
        X: numpy array of shape [n_samples, n_features]
           Training set.
        y: numpy array of shape [n_samples]
           Target values.
        optimizer: {"sgd", "rmsprop", "adagrad", "adadelta", "adam", "adamax",
                   "nadam"}, default='adam'
                   Optimizer
        lr: float>=0, default=0.001
            Learning rate.
        momentum: float>=0, default=0.0
                  Parameter updates momentum.
        nesterov: boolean, default=False
                  Whether to apply Nesterov momentum.
        decay: float>=0, default=0.0
               Learning rate decay over each update.
        rho: float>=0, default=0.9
        epsilon: float>=0, default=1e-08
                 Fuzz factor.
        beta_1: float in (0, 1), default=0.9
        beta_2: float in (0, 1), default=0.999
        schedule_decay: , default=0.004
        loss: string/function, default='mse'/'categorical_crossentropy'
              Loss function.
        metrics: list, default=None
                 List of metrics to be evaluated by the model during training
                 and testing.
        loss_weights: list or dictionary, default=None
                      Scalar coefficients to weight the loss contributions of
                      different model outputs.
        sample_weight_mode: {"temporal", None}, default=None
                            Timestep-wise sample weighting.
        batch_size: integer, default='auto'
                    Number of samples per gradient update.
        epochs: integer, default=200
                The number of times to iterate over the training data arrays.
        verbose: {0, 1, 2}, default=1
                 Verbosity mode. 0=silent, 1=verbose, 2=one log line per epoch.
        early_stopping: bool, default True
                        Whether to use early stopping to terminate training
                        when validation score is not improving.
        tol: float, default 1e-4
             Tolerance for the optimization.
        patience: integer, default 2
                  Number of epochs with no improvement after which training will
                  be stopped.
        validation_split: float in [0, 1], default=0.1
                          Fraction of the training data to be used as validation
                          data.
        validation_data: array-like, shape ((n_samples, features_shape),
                                            (n_samples, targets_shape)),
                         default=None
                         Data on which to evaluate the loss and any model
                         metrics at the end of each epoch.
        shuffle: boolean, default=True
                 Whether to shuffle the training data before each epoch.
        class_weight: dictionary, default=None
                      class indices to weights to apply to the model's loss for
                      the samples from each class during training.
        sample_weight: array-like, shape (n_samples), default=None
                       Weights to apply to the model's loss for each sample.
        initial_epoch: integer, default=0
                       Epoch at which to start training.
        Returns
        -------
        self
        """
        for k, v in locals().items():
            if (k != 'self') and (v is not None): self.__dict__[k] = v
        X, y = check_X_y(X, y, allow_nd=True, multi_output=True)
        if len(y.shape) == 1: y = y.reshape((len(y), 1))
        X, y = _time_series(X, y=y, window=self.window,
                            return_sequences=self.return_sequences)
        self.model_ = Architecture(X, y, transformer=None,
                                   activation=self.activation,
                                   use_bias=self.use_bias,
                                   kernel_initializer=self.kernel_initializer,
                                   bias_initializer=self.bias_initializer,
                                   kernel_regularizer_l1=self.kernel_regularizer_l1,
                                   kernel_regularizer_l2=self.kernel_regularizer_l2,
                                   bias_regularizer_l1=self.bias_regularizer_l1,
                                   bias_regularizer_l2=self.bias_regularizer_l2,
                                   activity_regularizer_l1=self.activity_regularizer_l1,
                                   activity_regularizer_l2=self.activity_regularizer_l2,
                                   kernel_constraint=self.kernel_constraint,
                                   bias_constraint=self.bias_constraint,
                                   return_sequences=self.return_sequences)
        optimizer = Optimizer(optimizer=self.optimizer, lr=self.lr,
                              momentum=self.momentum, nesterov=self.nesterov,
                              decay=self.decay, rho=self.rho,
                              epsilon=self.epsilon, beta_1=self.beta_1,
                              beta_2=self.beta_2,
                              schedule_decay=self.schedule_decay)
        self.model_.compile(optimizer, self.loss, metrics=self.metrics,
                            loss_weights=self.loss_weights,
                            sample_weight_mode=self.sample_weight_mode)
        callbacks = [EarlyStopping(monitor='val_loss' if (self.validation_split > 0.0 or self.validation_data is not None) else 'loss',
                                   min_delta=self.tol, patience=self.patience)] if self.early_stopping and (self.tol > 0.0) else []
        self.history_ = self.model_.fit(X, y,
                                        batch_size=min(200, len(X)) if self.batch_size == 'auto' else self.batch_size,
                                        epochs=self.epochs,
                                        verbose=1,
                                        callbacks=callbacks,
                                        validation_split=self.validation_split,
                                        validation_data=self.validation_data,
                                        shuffle=self.shuffle,
                                        class_weight=self.class_weight,
                                        sample_weight=np.asarray(self.sample_weight) if type(self.sample_weight) in (list, tuple) else self.sample_weight,
                                        initial_epoch=self.initial_epoch)
        return self

    def predict(self, X, batch_size=32, verbose=0):
        """Predict using the trained model.
        Parameters
        ----------
        X: array-like, shape (n_samples, features_shape)
           The input data.
        batch_size: integer, default=32
                    Batch size.
        verbose: {0, 1}, default=0
                 Verbosity mode.
        Returns
        -------
        y_pred: array-like, shape (n_samples, targets_shape)
                Target predictions for X.
        """
        check_is_fitted(self, ['model_', 'history_'])
        X = check_array(X, allow_nd=True)
        X, _ = _time_series(X, y=None, window=self.window,
                            return_sequences=self.return_sequences)
        preds = self.model_.predict(X, batch_size=batch_size, verbose=verbose)
        return preds.reshape((len(preds))) if (len(preds.shape) == 2 and preds.shape[1] == 1) else preds

    def transform(self, X):
        """Transform using the trained model.
        Parameters
        ----------
        X: array-like, shape (n_samples, features_shape)
           The input data.
        Returns
        -------
        Z: array-like, shape (n_samples, last_hidden_layer_shape)
           Transformations for X.
        """
        check_is_fitted(self, ['model_', 'history_'])
        X = check_array(X, allow_nd=True)
        X, _ = _time_series(X, y=None, window=self.window,
                            return_sequences=self.return_sequences)
        propagate = K.function([self.model_.layers[0].input],
                               [self.model_.layers[-2].output])
        return propagate([X])[0]

    def score(self, X, y, sample_weight=None, metric=r2_score):
        """Return the score of the model on the data X.
        Parameters
        ----------
        X: array-like, shape (n_samples, features_shape)
           Test samples.
        y: array-like, shape (n_samples, targets_shape)
           Targets for X.
        sample_weight: array-like, shape [n_samples], default=None
                       Sample weights.
        metric: function, default=r2_score/accuracy_score
                Metric to be evaluated.
        Returns
        -------
        score: float
               r2_score/accuracy of self.predict(X) wrt. y.
        """
        check_is_fitted(self, ['model_', 'history_'])
        X, y = check_X_y(X, y, allow_nd=True, multi_output=True)
        if len(y.shape) == 1: y = y.reshape((len(y), 1))
        _, y = _time_series(X, y=y, window=self.window,
                            return_sequences=self.return_sequences)
        return metric(y, self.predict(X), sample_weight=sample_weight)

class FFRegressor(BaseFeedForward, RegressorMixin):

    __doc__ = BaseFeedForward.__doc__
    
class Straight(BaseEstimator):

    """Straight feed-forward architecture.
    Basic straight feed-forward model architecture.
    Parameters
    ----------
    convolution_filters: integer, default=None
                         Dimensionality of the output space.
    convolution_kernel_size: integer/tuple/list, default=None
                             Dimensionality of the convolution window.
    convolution_strides: integer/tuple/list, default=None
                         Strides of the convolution.
    convolution_padding: {"valid", "same"}, default='valid'
    convolution_dilation_rate: integer/tuple/list, default=None
                               Dilation rate to use for dilated convolution.
    convolution_activation: string/function, default=None
                            Activation function.
    convolution_use_bias: boolean, default=True
                          Whether the layer uses a bias vector.
    convolution_kernel_initializer: string/function, default='glorot_uniform'
                                    Initializer for the kernel weights matrix.
    convolution_bias_initializer: string/function, default='zeros'
                                  Initializer for the bias vector.
    convolution_kernel_constraint: function, default=None
                                   Constraint function applied to the kernel
                                   matrix.
    convolution_bias_constraint: function, default=None
                                 Constraint function applied to the bias vector.
    pooling_type: {"max", "average}, default='max'
    pooling_pool_size: integer/tuple/list, default=None
                       Factors by which to downscale.
    pooling_strides: integer/tuple/list, default=None
                     Strides values.
    pooling_padding: {"valid", "same"}, default='valid'
    recurrent_type: {"lstm", "gru"}, default='lstm'
    recurrent_units: integer, default=None
                     Dimensionality of the output space.
    recurrent_activation: string/function, default='tanh'
                          Activation function to use.
    recurrent_recurrent_activation: string/function, default='hard_sigmoid'
                                    Activation function to use for the recurrent
                                    step.
    recurrent_use_bias: boolean, default=True
                        Whether the layer uses a bias vector.
    recurrent_kernel_initializer: string/function, default='glorot_uniform'
                                  Initializer for the kernel weights matrix.
    recurrent_recurrent_initializer: string/function, default='orthogonal'
                                     Initializer for the recurrent_kernel
                                     weights matrix.
    recurrent_bias_initializer: string/function, default='zeros'
                                Initializer for the bias vector.
    recurrent_unit_forget_bias: boolean, default=True
                                If True, add 1 to the bias of the forget gate
                                at initialization.
    recurrent_kernel_constraint: function, default=None
                                 Constraint function applied to the kernel
                                 weights matrix.
    recurrent_recurrent_constraint: function, default=None
                                    Constraint function applied to the
                                    recurrent_kernel weights matrix.
    recurrent_bias_constraint: function, default=None
                               Constraint function applied to the bias vector.
    recurrent_dropout: float in [0, 1], default=0.0
                       Fraction of the units to drop for the linear
                       transformation of the inputs.
    recurrent_recurrent_dropout: float in [0, 1], default=0.0
                                 Fraction of the units to drop for the linear
                                 transformation of the recurrent state.
    recurrent_return_sequences: boolean, default=False
                                Whether to return the last output in the output
                                sequence, or the full sequence.
    recurrent_go_backwards: boolean, default=False
                            If True, process the input sequence backwards and
                            return the reversed sequence.
    recurrent_stateful: boolean, default=False
                        If True, the last state for each sample at index i in a
                        batch will be used as initial state for the sample of
                        index i in the following batch.
    recurrent_unroll: boolean, default=False
                      If True, the network will be unrolled, else a symbolic
                      loop will be used.
    recurrent_implementation: {0, 1, 2}, default=0
    batchnormalization: boolean, default=False
                        Whether to perform batch normalization or not.
    batchnormalization_axis: integer, default=-1
                             The axis that should be normalized (typically the
                             features axis).
    batchnormalization_momentum: float, default=0.99
                                 Momentum for the moving average.
    batchnormalization_epsilon: float, default=0.001
                                Small float added to variance to avoid dividing
                                by zero.
    batchnormalization_center: boolean, default=True
                               If True, add offset of beta to normalized tensor.
                               If False, beta is ignored.
    batchnormalization_scale: boolean, default=True
                              If True, multiply by gamma. If False, gamma is not
                              used.
    batchnormalization_beta_initializer: string/function, default='zeros'
                                         Initializer for the beta weight.
    batchnormalization_gamma_initializer: string/function, default='ones'
                                          Initializer for the gamma weight.
    batchnormalization_moving_mean_initializer: string/function, default='zeros'
                                                Initializer for the moving mean.
    batchnormalization_moving_variance_initializer: string/function,
                                                    default='ones'
                                                    Initializer for the moving
                                                    variance.
    batchnormalization_beta_constraint: function, default=None
                                        Optional constraint for the beta weight.
    batchnormalization_gamma_constraint: function, default=None
                                         Optional constraint for the gamma
                                         weight.
    dense_units: integer, default=None
                 Dimensionality of the output space.
    dense_activation: string/function, default='relu'
                      Activation function to use.
    dense_use_bias: boolean, default=True
                    Whether the layer uses a bias vector.
    dense_kernel_initializer: string/function, default='he_uniform'
                              Initializer for the kernel weights matrix.
    dense_bias_initializer: string/function, default='zeros'
                            Initializer for the bias vector.
    dense_kernel_constraint: function, default=None
                             Constraint function applied to the kernel weights
                             matrix.
    dense_bias_constraint: function, default=None
                           Constraint function applied to the bias vector.
    dropout_rate: float in [0, 1], default=0.0
                  Fraction of the input units to drop.
    dropout_noise_shape: array-like, default=None
                         shape of the binary dropout mask that will be
                         multiplied with the input.
    dropout_seed: integer, default=None
                  Random seed.
    kernel_regularizer_l1: float, default=None
                           L1 regularization factor applied to the kernel
                           weights matrix.
    kernel_regularizer_l2: float, default=None
                           L2 regularization factor applied to the kernel
                           weights matrix.
    bias_regularizer_l1: float, default=None
                         L1 regularization factor applied to the bias vector.
    bias_regularizer_l2: float, default=None
                         L2 regularization factor applied to the bias vector.
    activity_regularizer_l1: float, default=None
                             L1 regularization factor applied to the output of
                             the layer.
    activity_regularizer_l2: float, default=None
                             L2 regularization factor applied to the output of
                             the layer.
    recurrent_regularizer_l1: float, default=None
                              L1 regularization factor applied to the
                              recurrent_kernel weights matrix.
    recurrent_regularizer_l2: float, default=None
                              L2 regularization factor applied to the
                              recurrent_kernel weights matrix.
    beta_regularizer_l1: float, default=None
                         L1 regularization factor applied to the beta weight.
    beta_regularizer_l2: float, default=None
                         L2 regularization factor applied to the beta weight.
    gamma_regularizer_l1: float, default=None
                          L1 regularization factor applied to the gamma  weight.
    gamma_regularizer_l2: float, default=None
                          L2 regularization factor applied to the gamma  weight.
    """

    def __init__(self, convolution_filters=None, convolution_kernel_size=None,
                 convolution_strides=None, convolution_padding='valid',
                 convolution_dilation_rate=None, convolution_activation=None,
                 convolution_use_bias=True,
                 convolution_kernel_initializer='glorot_uniform',
                 convolution_bias_initializer='zeros',
                 convolution_kernel_constraint=None,
                 convolution_bias_constraint=None, pooling_type='max',
                 pooling_pool_size=None, pooling_strides=None,
                 pooling_padding='valid', recurrent_type='lstm',
                 recurrent_units=None, recurrent_activation='tanh',
                 recurrent_recurrent_activation='hard_sigmoid',
                 recurrent_use_bias=True,
                 recurrent_kernel_initializer='glorot_uniform',
                 recurrent_recurrent_initializer='orthogonal',
                 recurrent_bias_initializer='zeros',
                 recurrent_unit_forget_bias=True,
                 recurrent_kernel_constraint=None,
                 recurrent_recurrent_constraint=None,
                 recurrent_bias_constraint=None, recurrent_dropout=0.0,
                 recurrent_recurrent_dropout=0.0,
                 recurrent_return_sequences=False, recurrent_go_backwards=False,
                 recurrent_stateful=False, recurrent_unroll=False,
                 recurrent_implementation=0, batchnormalization=False,
                 batchnormalization_axis=-1, batchnormalization_momentum=0.99,
                 batchnormalization_epsilon=0.001,
                 batchnormalization_center=True, batchnormalization_scale=True,
                 batchnormalization_beta_initializer='zeros',
                 batchnormalization_gamma_initializer='ones',
                 batchnormalization_moving_mean_initializer='zeros',
                 batchnormalization_moving_variance_initializer='ones',
                 batchnormalization_beta_constraint=None,
                 batchnormalization_gamma_constraint=None, dense_units=None,
                 dense_activation='relu', dense_use_bias=True,
                 dense_kernel_initializer='he_uniform',
                 dense_bias_initializer='zeros', kernel_regularizer_l1=None,
                 kernel_regularizer_l2=None, bias_regularizer_l1=None,
                 bias_regularizer_l2=None, activity_regularizer_l1=None,
                 activity_regularizer_l2=None, recurrent_regularizer_l1=None,
                 recurrent_regularizer_l2=None, beta_regularizer_l1=None,
                 beta_regularizer_l2=None, gamma_regularizer_l1=None,
                 gamma_regularizer_l2=None, dense_kernel_constraint=None,
                 dense_bias_constraint=None, dropout_rate=0.0,
                 dropout_noise_shape=None, dropout_seed=None):
        for k, v in locals().items():
            if k != 'self': self.__dict__[k] = v

    def _convolve_and_pool(self, x, convolution_filters,
                           convolution_kernel_size, convolution_strides,
                           convolution_dilation_rate, pooling_pool_size,
                           pooling_strides, return_tensors=True,
                           return_sequences=False):
        if convolution_kernel_size is not None:
            conv = {1: Conv1D, 2: Conv2D, 3: Conv3D}
            layer = conv[len(convolution_kernel_size)](convolution_filters, convolution_kernel_size,
                                                       strides=convolution_strides,
                                                       padding=self.convolution_padding,
                                                       dilation_rate=convolution_dilation_rate,
                                                       activation=self.convolution_activation,
                                                       use_bias=self.convolution_use_bias,
                                                       kernel_initializer=self.convolution_kernel_initializer,
                                                       bias_initializer=self.convolution_bias_initializer,
                                                       kernel_regularizer=self._kernel_regularizer,
                                                       bias_regularizer=self._bias_regularizer,
                                                       activity_regularizer=self._activity_regularizer,
                                                       kernel_constraint=self.convolution_kernel_constraint,
                                                       bias_constraint=self.convolution_bias_constraint)
            if return_sequences: layer = TimeDistributed(layer)
            x = layer(x)
        if pooling_pool_size is not None:
            pool = {'max': {1: MaxPooling1D, 2: MaxPooling2D, 3: MaxPooling3D},
                    'average': {1: AveragePooling1D, 2: AveragePooling2D,
                                3: AveragePooling3D}}
            layer = pool[self.pooling_type][len(pooling_pool_size)](pool_size=pooling_pool_size,
                                                                    strides=pooling_strides,
                                                                    padding=self.pooling_padding)
            if return_sequences: layer = TimeDistributed(layer)
            x = layer(x)
        if not return_tensors:
            layer = Flatten()
            if return_sequences: layer = TimeDistributed(layer)
            x = layer(x)
        return x

    def _recur(self, x, units, return_sequences=True):
        recur = {'lstm': LSTM, 'gru': GRU}
        layer = recur[self.recurrent_type](units, activation=self.recurrent_activation,
                                           recurrent_activation=self.recurrent_recurrent_activation,
                                           use_bias=self.recurrent_use_bias,
                                           kernel_initializer=self.recurrent_kernel_initializer,
                                           recurrent_initializer=self.recurrent_recurrent_initializer,
                                           bias_initializer=self.recurrent_bias_initializer,
                                           unit_forget_bias=self.recurrent_unit_forget_bias,
                                           kernel_regularizer=self._kernel_regularizer,
                                           recurrent_regularizer=self._recurrent_regularizer,
                                           bias_regularizer=self._bias_regularizer,
                                           activity_regularizer=self._activity_regularizer,
                                           kernel_constraint=self.recurrent_kernel_constraint,
                                           recurrent_constraint=self.recurrent_recurrent_constraint,
                                           bias_constraint=self.recurrent_bias_constraint,
                                           dropout=self.recurrent_dropout,
                                           recurrent_dropout=self.recurrent_recurrent_dropout,
                                           return_sequences=return_sequences,
                                           go_backwards=self.recurrent_go_backwards,
                                           stateful=self.recurrent_stateful,
                                           unroll=self.recurrent_unroll,
                                           implementation=self.recurrent_implementation)
        x = layer(x)
        return x

    def _connect(self, x, units, dropout_noise_shape=None):
        if self.batchnormalization:
            layer= BatchNormalization(axis=self.batchnormalization_axis,
                                      momentum=self.batchnormalization_momentum,
                                      epsilon=self.batchnormalization_epsilon,
                                      center=self.batchnormalization_center,
                                      scale=self.batchnormalization_scale,
                                      beta_initializer=self.batchnormalization_beta_initializer,
                                      gamma_initializer=self.batchnormalization_gamma_initializer,
                                      moving_mean_initializer=self.batchnormalization_moving_mean_initializer,
                                      moving_variance_initializer=self.batchnormalization_moving_variance_initializer,
                                      beta_regularizer=self._beta_regularizer,
                                      gamma_regularizer=self._gamma_regularizer,
                                      beta_constraint=self.batchnormalization_beta_constraint,
                                      gamma_constraint=self.batchnormalization_gamma_constraint)
            if self.recurrent_return_sequences: layer = TimeDistributed(layer)
            x = layer(x)
        layer = Dense(units, activation=self.dense_activation,
                      use_bias=self.dense_use_bias,
                      kernel_initializer=self.dense_kernel_initializer,
                      bias_initializer=self.dense_bias_initializer,
                      kernel_regularizer=self._kernel_regularizer,
                      bias_regularizer=self._bias_regularizer,
                      activity_regularizer=self._activity_regularizer,
                      kernel_constraint=self.dense_kernel_constraint,
                      bias_constraint=self.dense_bias_constraint)
        if self.recurrent_return_sequences: layer = TimeDistributed(layer)
        x = layer(x)
        if 0.0 < self.dropout_rate < 1.0:
            layer = Dropout(self.dropout_rate, noise_shape=dropout_noise_shape,
                            seed=self.dropout_seed)
            if self.recurrent_return_sequences: layer = TimeDistributed(layer)
            x = layer(x)
        return x

    def __call__(self, z):
        self._kernel_regularizer = Regularizer(l1=self.kernel_regularizer_l1,
                                               l2=self.kernel_regularizer_l2)
        self._bias_regularizer = Regularizer(l1=self.bias_regularizer_l1,
                                             l2=self.bias_regularizer_l2)
        self._activity_regularizer = Regularizer(l1=self.activity_regularizer_l1,
                                                 l2=self.activity_regularizer_l2)
        self._recurrent_regularizer = Regularizer(l1=self.recurrent_regularizer_l1,
                                                  l2=self.recurrent_regularizer_l2)
        self._beta_regularizer = Regularizer(l1=self.beta_regularizer_l1,
                                             l2=self.beta_regularizer_l2)
        self._gamma_regularizer = Regularizer(l1=self.gamma_regularizer_l1,
                                              l2=self.gamma_regularizer_l2)
        if (self.convolution_filters is not None) or (self.convolution_kernel_size is not None):
            if len(self.convolution_filters) == len(self.convolution_kernel_size):
                if self.convolution_strides is None: self.convolution_strides = [[1] * len(k) for k in self.convolution_kernel_size]
                if self.convolution_dilation_rate is None: self.convolution_dilation_rate = [[1] * len(k) for k in self.convolution_kernel_size]
                if self.pooling_pool_size is None: self.pooling_pool_size = [None] * len(self.convolution_filters)
                if self.pooling_strides is None: self.pooling_strides = [None] * len(self.convolution_filters)
                for i, (cf, cks, cs, cdr, pps, ps) in enumerate(zip(self.convolution_filters,
                                                                    self.convolution_kernel_size,
                                                                    self.convolution_strides,
                                                                    self.convolution_dilation_rate,
                                                                    self.pooling_pool_size,
                                                                    self.pooling_strides)):
                    z = self._convolve_and_pool(z, cf, cks, cs, cdr, pps, ps,
                                                return_tensors=i < len(self.convolution_filters) - 1,
                                                return_sequences=self.recurrent_units is not None)
        if self.recurrent_units is not None:
            for i, ru in enumerate(self.recurrent_units):
                z = self._recur(z, ru,
                                return_sequences=i < len(self.recurrent_units) - 1)
        if self.dense_units is not None:
            if self.dropout_noise_shape is None: self.dropout_noise_shape = [None] * len(self.dense_units)
            for (du, dns) in zip(self.dense_units, self.dropout_noise_shape):
                z = self._connect(z, du, dropout_noise_shape=dns)
        return z
    
class DSVR(FFRegressor):

    def __init__(self, architecture=None, activation='linear', use_bias=True,
                 kernel_initializer='glorot_uniform', bias_initializer='zeros',
                 kernel_regularizer_l1=None, kernel_regularizer_l2=None,
                 bias_regularizer_l1=None, bias_regularizer_l2=None,
                 activity_regularizer_l1=None, activity_regularizer_l2=None,
                 kernel_constraint=None, bias_constraint=None, optimizer='adam',
                 lr=0.001, momentum=0.0, nesterov=False, decay=0.0, rho=0.9,
                 epsilon=1e-08, beta_1=0.9, beta_2=0.999, schedule_decay=0.004,
                 loss='mse', metrics=None, loss_weights=None,
                 sample_weight_mode=None, batch_size='auto', epochs=200,
                 verbose=2, early_stopping=True, tol=0.0001, patience=2,
                 validation_split=0.1, validation_data=None, shuffle=True,
                 class_weight=None, sample_weight=None, initial_epoch=0,
                 window=None, return_sequences=False, loss_epsilon=0.1):
        for k, v in locals().items():
            if k != 'self':
                self.__dict__[k] = v

    def _epsilon_insensitive(self, y_true, y_pred):
        return K.mean(K.maximum(K.abs(y_pred - y_true) - self.loss_epsilon,
                                0.0), axis=-1)

    def fit(self, X, y, **kwargs):
        return FFRegressor.fit(self, X, y, loss=self._epsilon_insensitive,
                               **kwargs)
    
def fetch_file(dataname, urlname, data_home=None):
    """Fetch dataset.

    Fetch a file from a given url and stores it in a given directory.

    Parameters
    ----------
    dataname: string
              Dataset name.
    urlname: string
             Dataset url.
    data_home: string, default=None
               Dataset directory.

    Returns
    -------
    filename: string
              Name of the file.

    """
    # check if this data set has been already downloaded
    data_home = get_data_home(data_home=data_home)
    data_home = join(data_home, dataname)
    if not exists(data_home):
        makedirs(data_home)
    filename = join(data_home, basename(normpath(urlname)))
    # if the file does not exist, download it
    if not exists(filename):
        try:
            data_url = urlopen(urlname)
        except HTTPError as e:
            if e.code == 404:
                e.msg = "Dataset '%s' not found." % dataname
            raise
        # store file
        try:
            with open(filename, 'w+b') as data_file:
                copyfileobj(data_url, data_file)
        except:
            remove(filename)
            raise
        data_url.close()
    return filename

def load_train_scale(name, url, url_scale, fetch_file=fetch_file,
                     return_X_y=False):
    """Load dataset with scaled version.

    Load a dataset with scaled version.

    Parameters
    ----------
    name: string
          Dataset name.
    url: string
         Dataset url.
    url_scale: string
               Scaled dataset url.
    fetch_file: function, default=fetch_file
                Dataset fetching function.
    return_X_y: bool, default=False
                If True, returns (data, target) instead of a Bunch object..

    Returns
    -------
    data: Bunch
          Dictionary-like object with all the data and metadata.
    X, y: arrays
          If return_X_y is True

    """
    filename = fetch_file(name, url)
    filename_scale = fetch_file(name, url_scale)
    X, y, X_scale, y_scale = load_svmlight_files([filename, filename_scale])
    X = X.todense()
    X_scale = X_scale.todense()

    if return_X_y:
        return X, y

    return Bunch(data=X, target=y, data_scale=X_scale, target_scale=y_scale)

## Global Variables

In [4]:
perc_train=0.7;
perc_val=0.2;
cv=3;

## Load Dataset

In [5]:
if (dataset=='housing'):
    X,y=load_train_scale('housing',
                            'https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression/housing',
                            'https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression/housing_scale',
                            return_X_y=True)
    X=pd.DataFrame(X);
    X["y"]=y;
elif (dataset=='space_ga'):
    X,y=load_train_scale('space_ga',
                                'https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression/space_ga',
                                'https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression/space_ga_scale',
                                return_X_y=True)
    X=pd.DataFrame(X);
    X["y"]=y;



In [6]:
X=shuffle(X);
print(X.shape)
print(X.head())

(506, 14)
            0    1      2    3      4      5     6       7     8      9    10  \
155   3.53501  0.0  19.58  1.0  0.871  6.152  82.6  1.7455   5.0  403.0  14.7   
313   0.26938  0.0   9.90  0.0  0.544  6.266  82.8  3.2628   4.0  304.0  18.4   
477  15.02340  0.0  18.10  0.0  0.614  5.304  97.3  2.1007  24.0  666.0  20.2   
175   0.06664  0.0   4.05  0.0  0.510  6.546  33.1  3.1323   5.0  296.0  16.6   
44    0.12269  0.0   6.91  0.0  0.448  6.069  40.0  5.7209   3.0  233.0  17.9   

         11     12     y  
155   88.01  15.02  15.6  
313  393.39   7.90  21.6  
477  349.48  24.91  12.0  
175  390.96   5.33  29.4  
44   389.39   9.55  21.2  


## train/validation/test split

In [7]:
X_train=X[0:int(perc_train*X.shape[0])]
X_val=X[(int(perc_train*X.shape[0])):(int((perc_train+perc_val)*X.shape[0]))]
X_test=X[(int((perc_train+perc_val)*X.shape[0])):]
y_train=X_train["y"];
y_val=X_val["y"];
y_test=X_test["y"];
X=X.drop(columns=['y']);
X_train=X_train.drop(columns=['y']);
X_val=X_val.drop(columns=['y']);
X_test=X_test.drop(columns=['y']);

print(X_train.shape);
print(X_val.shape);
print(X_test.shape);
print(y_train.shape);
print(y_val.shape);
print(y_test.shape);

(354, 13)
(101, 13)
(51, 13)
(354,)
(101,)
(51,)


## Standard Neural Net

In [8]:
hidden_layer_sizes_values=[5,7,10];
learning_rate_init_values=[np.float_power(10,-10),0.0001,0.001,0.01,0.1];
val_errors=pd.DataFrame();
for hidden_layer_sizes in hidden_layer_sizes_values:
    for learning_rate_init in learning_rate_init_values:
        clf = MLPRegressor(solver='sgd', 
                            hidden_layer_sizes=hidden_layer_sizes,learning_rate_init=learning_rate_init,
                            alpha=1e-5,random_state=1);
        clf.fit(X_train, y_train);
        mae=np.mean(np.absolute(clf.predict(X_val)-y_val))
        val_errors = val_errors.append(pd.DataFrame(data={'hidden_layer_sizes':[hidden_layer_sizes],'learning_rate_init':[learning_rate_init],'score':[mae]}), ignore_index=True)
val_errors=val_errors.sort_values(['score'])

/home/falendor/.conda/envs/tensorflow/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [9]:
val_errors

,hidden_layer_sizes,learning_rate_init,score
2,5,1.000000e-03,6.492090e+00
4,5,1.000000e-01,7.822429e+00
3,5,1.000000e-02,8.017087e+00
1,5,1.000000e-04,1.570930e+01
6,7,1.000000e-04,2.427472e+01
5,7,1.000000e-10,3.974522e+01
11,10,1.000000e-04,5.629462e+01
0,5,1.000000e-10,1.742959e+02
10,10,1.000000e-10,2.232732e+02
7,7,1.000000e-03,9.621949e+02


In [10]:
clf = MLPRegressor(solver='sgd', 
                            hidden_layer_sizes=val_errors["hidden_layer_sizes"].iloc[0],
                            learning_rate_init=val_errors["learning_rate_init"].iloc[0],
                            alpha=1e-5,random_state=1);
clf.fit(X_train, y_train);
MLP_scores=np.mean(np.absolute(clf.predict(X_test)-y_test))
MLP_scores

/home/falendor/.conda/envs/tensorflow/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


9.1344008472462068

## SVR and DeepSVR

### Combine Train and Validation (we will use CV from now on)

In [11]:
X_train=X_train.append(X_val);
y_train=y_train.append(y_val);
print(X_train.shape);
print(y_train.shape);

(455, 13)
(455,)


In [12]:
print(X_train.head())
print(y_train[0:11])

           0    1      2    3      4      5     6       7     8      9     10  \
155   3.53501  0.0  19.58  1.0  0.871  6.152  82.6  1.7455   5.0  403.0  14.7   
313   0.26938  0.0   9.90  0.0  0.544  6.266  82.8  3.2628   4.0  304.0  18.4   
477  15.02340  0.0  18.10  0.0  0.614  5.304  97.3  2.1007  24.0  666.0  20.2   
175   0.06664  0.0   4.05  0.0  0.510  6.546  33.1  3.1323   5.0  296.0  16.6   
44    0.12269  0.0   6.91  0.0  0.448  6.069  40.0  5.7209   3.0  233.0  17.9   

         11     12  
155   88.01  15.02  
313  393.39   7.90  
477  349.48  24.91  
175  390.96   5.33  
44   389.39   9.55  
155    15.6
313    21.6
477    12.0
175    29.4
44     21.2
51     20.5
341    32.7
142    13.4
10     15.0
503    23.9
389    11.5
Name: y, dtype: float64


### Define Model Parameters

In [13]:
epochs=1000;
n_hidden=100;
scoring = {'mean_absolute_error': make_scorer(mean_absolute_error)};
sigma=np.std(y_train);

### Define Model Hyperparameters (Grid)

In [14]:
learning_rate=[0.01];
cs = np.float_power(10,np.arange(-3,-2));
epsilons = sigma * np.logspace(-6, 3, base=2.0,num=1);
gammas = np.logspace(-3, 6, base=10.0,num=1);
print(learning_rate);
print(cs);
print(epsilons);
print(gammas);

[0.01]
[ 0.001]
[ 0.13784907]
[ 0.001]


### Define Models

In [15]:
dsvr = lambda n: GridSearchCV(Pipeline([('scaler', StandardScaler()), ('regressor', DSVR(architecture=Straight(dense_units=[n_hidden]*n), epochs=epochs))]),
                                   {'regressor__kernel_regularizer_l2': cs,
                                    'regressor__lr': learning_rate,
                                    'regressor__loss_epsilon': epsilons},
                                   scoring=scoring['mean_absolute_error'], cv=cv, error_score=np.nan, fit_params={'regressor__epochs': epochs})
 


estimator = {'SVR': GridSearchCV(Pipeline([('scaler', StandardScaler()), ('regressor', SVR())]),
                                      {'regressor__C': cs,
                                       'regressor__epsilon': epsilons,
                                       'regressor__gamma': gammas},
                                      scoring=scoring['mean_absolute_error'], cv=cv, error_score=np.nan),
                 'DSVR0': dsvr(0), 'DSVR1': dsvr(1), 'DSVR2': dsvr(2), 'DSVR3': dsvr(3), 'DSVR4': dsvr(4), 'DSVR5': dsvr(5)}   

### Train SVR Model

In [16]:
selected_estimator=estimator['SVR'];
selected_estimator.fit(X_train,y_train);
SVR_scores = selected_estimator.score(X_test, y_test);

### Train DeepSVR Model

In [17]:
selected_estimator=estimator['DSVR5'];
selected_estimator.fit(X_train,y_train);
DeepSVR_scores = selected_estimator.score(X_test, y_test);

/home/falendor/.conda/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_search.py:584: DeprecationWarning: "fit_params" as a constructor argument was deprecated in version 0.19 and will be removed in version 0.21. Pass fit parameters to the "fit" method instead.
  '"fit" method instead.', DeprecationWarning)


Train on 272 samples, validate on 31 samples
Epoch 1/1000
272/272 [==============================] - 0s 317us/step - loss: 22.5322 - val_loss: 23.2261
Epoch 2/1000
272/272 [==============================] - 0s 23us/step - loss: 22.5080 - val_loss: 23.2167
Epoch 3/1000
272/272 [==============================] - 0s 19us/step - loss: 22.4867 - val_loss: 23.2080
Epoch 4/1000
272/272 [==============================] - 0s 19us/step - loss: 22.4655 - val_loss: 23.1957
Epoch 5/1000
272/272 [==============================] - 0s 21us/step - loss: 22.4443 - val_loss: 23.1815
Epoch 6/1000
272/272 [==============================] - 0s 20us/step - loss: 22.4235 - val_loss: 23.1666
Epoch 7/1000
272/272 [==============================] - 0s 15us/step - loss: 22.4029 - val_loss: 23.1509
Epoch 8/1000
272/272 [==============================] - 0s 17us/step - loss: 22.3824 - val_loss: 23.1366
Epoch 9/1000
272/272 [==============================] - 0s 19us/step - loss: 22.3617 - val_loss: 23.1240
Epoch 10/

272/272 [==============================] - 0s 19us/step - loss: 19.3582 - val_loss: 21.4156
Epoch 154/1000
272/272 [==============================] - 0s 26us/step - loss: 19.3381 - val_loss: 21.4006
Epoch 155/1000
272/272 [==============================] - 0s 25us/step - loss: 19.3177 - val_loss: 21.3864
Epoch 156/1000
272/272 [==============================] - 0s 21us/step - loss: 19.2969 - val_loss: 21.3724
Epoch 157/1000
272/272 [==============================] - 0s 19us/step - loss: 19.2765 - val_loss: 21.3587
Epoch 158/1000
272/272 [==============================] - 0s 27us/step - loss: 19.2559 - val_loss: 21.3443
Epoch 159/1000
272/272 [==============================] - 0s 22us/step - loss: 19.2360 - val_loss: 21.3296
Epoch 160/1000
272/272 [==============================] - 0s 20us/step - loss: 19.2151 - val_loss: 21.3145
Epoch 161/1000
272/272 [==============================] - 0s 19us/step - loss: 19.1948 - val_loss: 21.3011
Epoch 162/1000
272/272 [============================

272/272 [==============================] - 0s 23us/step - loss: 16.2920 - val_loss: 18.9215
Epoch 306/1000
272/272 [==============================] - 0s 27us/step - loss: 16.2719 - val_loss: 18.9081
Epoch 307/1000
272/272 [==============================] - 0s 32us/step - loss: 16.2522 - val_loss: 18.8942
Epoch 308/1000
272/272 [==============================] - 0s 34us/step - loss: 16.2326 - val_loss: 18.8794
Epoch 309/1000
272/272 [==============================] - 0s 32us/step - loss: 16.2125 - val_loss: 18.8639
Epoch 310/1000
272/272 [==============================] - 0s 29us/step - loss: 16.1926 - val_loss: 18.8487
Epoch 311/1000
272/272 [==============================] - 0s 32us/step - loss: 16.1727 - val_loss: 18.8328
Epoch 312/1000
272/272 [==============================] - 0s 29us/step - loss: 16.1529 - val_loss: 18.8174
Epoch 313/1000
272/272 [==============================] - 0s 26us/step - loss: 16.1326 - val_loss: 18.8030
Epoch 314/1000
272/272 [============================

272/272 [==============================] - 0s 20us/step - loss: 13.2525 - val_loss: 16.2481
Epoch 458/1000
272/272 [==============================] - 0s 23us/step - loss: 13.2331 - val_loss: 16.2326
Epoch 459/1000
272/272 [==============================] - 0s 28us/step - loss: 13.2133 - val_loss: 16.2168
Epoch 460/1000
272/272 [==============================] - 0s 24us/step - loss: 13.1941 - val_loss: 16.1996
Epoch 461/1000
272/272 [==============================] - 0s 18us/step - loss: 13.1750 - val_loss: 16.1825
Epoch 462/1000
272/272 [==============================] - 0s 23us/step - loss: 13.1551 - val_loss: 16.1657
Epoch 463/1000
272/272 [==============================] - 0s 21us/step - loss: 13.1362 - val_loss: 16.1495
Epoch 464/1000
272/272 [==============================] - 0s 25us/step - loss: 13.1166 - val_loss: 16.1342
Epoch 465/1000
272/272 [==============================] - 0s 16us/step - loss: 13.0967 - val_loss: 16.1194
Epoch 466/1000
272/272 [============================

...

409/409 [==============================] - 0s 17us/step - loss: 13.3046 - val_loss: 14.6095
Epoch 293/1000
409/409 [==============================] - 0s 22us/step - loss: 13.2733 - val_loss: 14.5882
Epoch 294/1000
409/409 [==============================] - 0s 29us/step - loss: 13.2428 - val_loss: 14.5678
Epoch 295/1000
409/409 [==============================] - 0s 24us/step - loss: 13.2101 - val_loss: 14.5459
Epoch 296/1000
409/409 [==============================] - 0s 21us/step - loss: 13.1794 - val_loss: 14.5207
Epoch 297/1000
409/409 [==============================] - 0s 27us/step - loss: 13.1480 - val_loss: 14.4973
Epoch 298/1000
409/409 [==============================] - 0s 20us/step - loss: 13.1180 - val_loss: 14.4717
Epoch 299/1000
409/409 [==============================] - 0s 19us/step - loss: 13.0871 - val_loss: 14.4419
Epoch 300/1000
409/409 [==============================] - 0s 33us/step - loss: 13.0575 - val_loss: 14.4117
Epoch 301/1000
409/409 [============================

409/409 [==============================] - 0s 18us/step - loss: 8.7865 - val_loss: 10.6840
Epoch 445/1000
409/409 [==============================] - 0s 22us/step - loss: 8.7581 - val_loss: 10.6549
Epoch 446/1000
409/409 [==============================] - 0s 24us/step - loss: 8.7293 - val_loss: 10.6248
Epoch 447/1000
409/409 [==============================] - 0s 17us/step - loss: 8.7007 - val_loss: 10.5964
Epoch 448/1000
409/409 [==============================] - 0s 27us/step - loss: 8.6722 - val_loss: 10.5687
Epoch 449/1000
409/409 [==============================] - 0s 25us/step - loss: 8.6442 - val_loss: 10.5403
Epoch 450/1000
409/409 [==============================] - 0s 23us/step - loss: 8.6163 - val_loss: 10.5110
Epoch 451/1000
409/409 [==============================] - 0s 21us/step - loss: 8.5886 - val_loss: 10.4774
Epoch 452/1000
409/409 [==============================] - 0s 25us/step - loss: 8.5609 - val_loss: 10.4434
Epoch 453/1000
409/409 [==============================] - 0s 

Epoch 522/1000
409/409 [==============================] - 0s 24us/step - loss: 6.6787 - val_loss: 8.5502
Epoch 523/1000
409/409 [==============================] - 0s 29us/step - loss: 6.6526 - val_loss: 8.5227
Epoch 524/1000
409/409 [==============================] - 0s 35us/step - loss: 6.6286 - val_loss: 8.4939
Epoch 525/1000
409/409 [==============================] - 0s 26us/step - loss: 6.6030 - val_loss: 8.4642
Epoch 526/1000
409/409 [==============================] - 0s 21us/step - loss: 6.5783 - val_loss: 8.4354
Epoch 527/1000
409/409 [==============================] - 0s 25us/step - loss: 6.5527 - val_loss: 8.4061
Epoch 528/1000
409/409 [==============================] - 0s 18us/step - loss: 6.5264 - val_loss: 8.3780
Epoch 529/1000
409/409 [==============================] - 0s 24us/step - loss: 6.4989 - val_loss: 8.3501
Epoch 530/1000
409/409 [==============================] - 0s 23us/step - loss: 6.4723 - val_loss: 8.3221
Epoch 531/1000
409/409 [==============================]

Epoch 600/1000
409/409 [==============================] - 0s 23us/step - loss: 4.7524 - val_loss: 6.5632
Epoch 601/1000
409/409 [==============================] - 0s 19us/step - loss: 4.7296 - val_loss: 6.5391
Epoch 602/1000
409/409 [==============================] - 0s 21us/step - loss: 4.7065 - val_loss: 6.5160
Epoch 603/1000
409/409 [==============================] - 0s 24us/step - loss: 4.6843 - val_loss: 6.4928
Epoch 604/1000
409/409 [==============================] - 0s 20us/step - loss: 4.6617 - val_loss: 6.4690
Epoch 605/1000
409/409 [==============================] - 0s 23us/step - loss: 4.6374 - val_loss: 6.4442
Epoch 606/1000
409/409 [==============================] - 0s 17us/step - loss: 4.6138 - val_loss: 6.4201
Epoch 607/1000
409/409 [==============================] - 0s 20us/step - loss: 4.5908 - val_loss: 6.3957
Epoch 608/1000
409/409 [==============================] - 0s 29us/step - loss: 4.5675 - val_loss: 6.3701
Epoch 609/1000
409/409 [==============================]

Epoch 755/1000
409/409 [==============================] - 0s 23us/step - loss: 3.0500 - val_loss: 4.3692
Epoch 756/1000
409/409 [==============================] - 0s 19us/step - loss: 3.0467 - val_loss: 4.3597
Epoch 757/1000
409/409 [==============================] - 0s 22us/step - loss: 3.0432 - val_loss: 4.3448
Epoch 758/1000
409/409 [==============================] - 0s 23us/step - loss: 3.0403 - val_loss: 4.3400
Epoch 759/1000
409/409 [==============================] - 0s 26us/step - loss: 3.0353 - val_loss: 4.3406
Epoch 760/1000
409/409 [==============================] - 0s 27us/step - loss: 3.0314 - val_loss: 4.3385
Epoch 761/1000
409/409 [==============================] - 0s 22us/step - loss: 3.0278 - val_loss: 4.3292
Epoch 762/1000
409/409 [==============================] - 0s 22us/step - loss: 3.0242 - val_loss: 4.3168
Epoch 763/1000
409/409 [==============================] - 0s 18us/step - loss: 3.0218 - val_loss: 4.3066
Epoch 764/1000
409/409 [==============================]

### Evaluate Models

In [18]:
print('MLP MAE: ' + str(MLP_scores));
print('SVR MAE: ' + str(SVR_scores));
print('DeepSVR MAE: ' + str(DeepSVR_scores));

MLP MAE: 9.13440084725
SVR MAE: 9.14771273212
DeepSVR MAE: 4.20074236627
